In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import small_rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from small_rubiks_neural_networks import student_network

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

my_student_network = student_network.load("models/trained.h5")
#my_student_network = student_network.create(residual_layers = 4)
student_template = student(task, 50,my_student_network)
t = teacher(setup, lambda n : 1+rng.poisson(lam=3,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10)
#c.run_training_batch(n_problems=1,epochs_per_episode=3)


In [3]:
for i in range(10):
    c.run_training_batch(n_problems=1,epochs_per_episode=3)
    #if i % 10 == 0:
    #    my_student_network.save("models/t.keras")

  OB    
  YB    
WOGOYYRB
WOGYRRWB
  WG    
  RG    

  OO    
  OO    
WWGGYYBB
WWGGYYBB
  RR    
  RR    

  OB    
  YB    
WOGOYYRB
WOGYRRWB
  WG    
  RG    

  OO    
  YY    
WOGGRYBB
WOGGRYBB
  WW    
  RR    

Finished step 0, 1 out of 1 remain open.
Finished step 1, 1 out of 1 remain open.
Finished step 2, 0 out of 1 remain open.
Finished step 3, 0 out of 1 remain open.
Finished step 4, 0 out of 1 remain open.
Finished step 5, 0 out of 1 remain open.
Finished step 6, 0 out of 1 remain open.
Finished step 7, 0 out of 1 remain open.
Finished step 8, 0 out of 1 remain open.
Finished step 9, 0 out of 1 remain open.
Epoch 1/3
1/1 [==============================] - 0s 14ms/step - loss: 1.5438 - eval_output_loss: 0.4924 - policy_output_loss: 0.0347
Epoch 2/3
1/1 [==============================] - 0s 11ms/step - loss: 1.2962 - eval_output_loss: 0.4100 - policy_output_loss: 0.0344
Epoch 3/3
1/1 [==============================] - 0s 9ms/step - loss: 1.0441 - eval_output_loss: 0.3260 -